In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

import tensorflow as tf

In [2]:
turbine_data= pd.read_csv('gas_turbines (1).csv')

In [3]:
turbine_data

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311
...,...,...,...,...,...,...,...,...,...,...,...
15034,9.0301,1005.6,98.460,3.5421,19.164,1049.7,546.21,111.61,10.400,4.5186,79.559
15035,7.8879,1005.9,99.093,3.5059,19.414,1046.3,543.22,111.78,10.433,4.8470,79.917
15036,7.2647,1006.3,99.496,3.4770,19.530,1037.7,537.32,110.19,10.483,7.9632,90.912
15037,7.0060,1006.8,99.008,3.4486,19.377,1043.2,541.24,110.74,10.533,6.2494,93.227


In [4]:
turbine_data.head()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311


In [5]:
turbine_data.isna().sum()

AT      0
AP      0
AH      0
AFDP    0
GTEP    0
TIT     0
TAT     0
TEY     0
CDP     0
CO      0
NOX     0
dtype: int64

In [6]:
turbine_data.columns

Index(['AT', 'AP', 'AH', 'AFDP', 'GTEP', 'TIT', 'TAT', 'TEY', 'CDP', 'CO',
       'NOX'],
      dtype='object')

In [7]:
turbine_data.describe()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
count,15039.000000,15039.00000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000
mean,17.764381,1013.19924,79.124174,4.200294,25.419061,1083.798770,545.396183,134.188464,12.102353,1.972499,68.190934
std,7.574323,6.41076,13.793439,0.760197,4.173916,16.527806,7.866803,15.829717,1.103196,2.222206,10.470586
min,0.522300,985.85000,30.344000,2.087400,17.878000,1000.800000,512.450000,100.170000,9.904400,0.000388,27.765000
25%,11.408000,1008.90000,69.750000,3.723900,23.294000,1079.600000,542.170000,127.985000,11.622000,0.858055,61.303500
50%,18.186000,1012.80000,82.266000,4.186200,25.082000,1088.700000,549.890000,133.780000,12.025000,1.390200,66.601000
75%,23.862500,1016.90000,90.043500,4.550900,27.184000,1096.000000,550.060000,140.895000,12.578000,2.160400,73.935500
max,34.929000,1034.20000,100.200000,7.610600,37.402000,1100.800000,550.610000,174.610000,15.081000,44.103000,119.890000


In [8]:
x= turbine_data[['AT', 'AP', 'AH', 'AFDP', 'GTEP', 'TIT', 'TAT', 'CDP', 'CO', 'NOX']]

In [9]:
x

,AT,AP,AH,AFDP,GTEP,TIT,TAT,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,10.612,3.2484,82.311
...,...,...,...,...,...,...,...,...,...,...
15034,9.0301,1005.6,98.460,3.5421,19.164,1049.7,546.21,10.400,4.5186,79.559
15035,7.8879,1005.9,99.093,3.5059,19.414,1046.3,543.22,10.433,4.8470,79.917
15036,7.2647,1006.3,99.496,3.4770,19.530,1037.7,537.32,10.483,7.9632,90.912
15037,7.0060,1006.8,99.008,3.4486,19.377,1043.2,541.24,10.533,6.2494,93.227


In [10]:
y= turbine_data['TEY']

In [11]:
y

0        114.70
1        114.72
2        114.71
3        114.72
4        114.72
          ...  
15034    111.61
15035    111.78
15036    110.19
15037    110.74
15038    111.58
Name: TEY, Length: 15039, dtype: float64

TUNING : BATCH SIZE AND EPOCH

In [12]:
from sklearn.model_selection import GridSearchCV, KFold
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam

In [13]:
# create a sequential model
def create_model():
  model= Sequential()
  model.add(Dense(10, input_dim=10, kernel_initializer= 'normal', activation='relu'))
  model.add(Dense(8, kernel_initializer= 'normal', activation='relu'))
  model.add(Dense(8, kernel_initializer= 'normal', activation='relu'))
  model.add(Dense(1, kernel_initializer= 'normal')) 
  adam= Adam()
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model

In [14]:
model= KerasRegressor(build_fn=create_model, verbose=0)
# create grid search model
batch_size=[50,100, 200, 400, 800, 1000]
epochs=[10,50,100]

param_grid= dict(batch_size= batch_size, epochs=epochs)
grid= GridSearchCV(estimator=model, param_grid=param_grid, cv= KFold(), verbose=10 )
grid_result= grid.fit(x,y)

C:\Users\shiji\AppData\Local\Temp\ipykernel_17596\4158208720.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model= KerasRegressor(build_fn=create_model, verbose=0)


Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5; 1/18] START batch_size=50, epochs=10...................................
[CV 1/5; 1/18] END .batch_size=50, epochs=10;, score=-240.642 total time=   7.5s
[CV 2/5; 1/18] START batch_size=50, epochs=10...................................
[CV 2/5; 1/18] END ..batch_size=50, epochs=10;, score=-93.299 total time=   6.2s
[CV 3/5; 1/18] START batch_size=50, epochs=10...................................
[CV 3/5; 1/18] END .batch_size=50, epochs=10;, score=-193.932 total time=   6.3s
[CV 4/5; 1/18] START batch_size=50, epochs=10...................................
[CV 4/5; 1/18] END .batch_size=50, epochs=10;, score=-165.387 total time=   6.5s
[CV 5/5; 1/18] START batch_size=50, epochs=10...................................
[CV 5/5; 1/18] END ..batch_size=50, epochs=10;, score=-99.404 total time=   8.0s
[CV 1/5; 2/18] START batch_size=50, epochs=50...................................
[CV 1/5; 2/18] END ..batch_size=50, epochs=50;, 

[CV 1/5; 11/18] END batch_size=400, epochs=50;, score=-266.997 total time=   3.3s
[CV 2/5; 11/18] START batch_size=400, epochs=50.................................
[CV 2/5; 11/18] END batch_size=400, epochs=50;, score=-87.451 total time=   3.7s
[CV 3/5; 11/18] START batch_size=400, epochs=50.................................
[CV 3/5; 11/18] END batch_size=400, epochs=50;, score=-272.473 total time=   3.4s
[CV 4/5; 11/18] START batch_size=400, epochs=50.................................
[CV 4/5; 11/18] END batch_size=400, epochs=50;, score=-191.359 total time=   3.5s
[CV 5/5; 11/18] START batch_size=400, epochs=50.................................
[CV 5/5; 11/18] END batch_size=400, epochs=50;, score=-211.387 total time=   3.7s
[CV 1/5; 12/18] START batch_size=400, epochs=100................................
[CV 1/5; 12/18] END batch_size=400, epochs=100;, score=-17974.260 total time=   6.1s
[CV 2/5; 12/18] START batch_size=400, epochs=100................................
[CV 2/5; 12/18] END 

In [15]:
# results
print('Best:{}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means= grid_result.cv_results_['mean_test_score']
stds= grid_result.cv_results_['std_test_score']
params= grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best:-1.9347431659698486, using {'batch_size': 50, 'epochs': 100}
-158.53265533447265,56.20267673479764 with: {'batch_size': 50, 'epochs': 10}
-4.207967948913574,4.015816982615523 with: {'batch_size': 50, 'epochs': 50}
-1.9347431659698486,1.063049625154314 with: {'batch_size': 50, 'epochs': 100}
-205.42647247314454,65.28919734740983 with: {'batch_size': 100, 'epochs': 10}
-6.197259712219238,2.2560073371933598 with: {'batch_size': 100, 'epochs': 50}
-2.5201219081878663,1.944828877367256 with: {'batch_size': 100, 'epochs': 100}
-4033.4645965576174,7634.308796091807 with: {'batch_size': 200, 'epochs': 10}
-66.29109191894531,35.93215363950301 with: {'batch_size': 200, 'epochs': 50}
-4.3565103769302365,1.93490297341826 with: {'batch_size': 200, 'epochs': 100}
-235.45767211914062,69.18607533139553 with: {'batch_size': 400, 'epochs': 10}
-205.93348541259766,66.98726473748086 with: {'batch_size': 400, 'epochs': 50}
-3639.160053062439,7167.795571680803 with: {'batch_size': 400, 'epochs': 100}
-

In [16]:
from keras.layers import Dropout

# defining model
def create_model_lr_drate(learning_rate,dropout_rate):
  model= Sequential()
  model.add(Dense(10, input_dim=10, kernel_initializer= 'normal', activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(8, kernel_initializer= 'normal', activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(8, kernel_initializer= 'normal', activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(1, kernel_initializer= 'normal')) 
  adam= Adam(learning_rate= learning_rate)
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model

# Define regression model
model= KerasRegressor(build_fn=create_model_lr_drate, verbose=0, batch_size=100, epochs=100)
learning_rate=[0.001, 0.01, 0.1, 0.2, 0.5]
dropout_rate=[0, 0.1, 0.2]

param_grid= dict(learning_rate= learning_rate, dropout_rate=dropout_rate)
grid= GridSearchCV(estimator=model, param_grid=param_grid, cv= KFold(), verbose=10 )
grid_result= grid.fit(x,y)


Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5; 1/15] START dropout_rate=0, learning_rate=0.001........................


C:\Users\shiji\AppData\Local\Temp\ipykernel_17596\2203052668.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model= KerasRegressor(build_fn=create_model_lr_drate, verbose=0, batch_size=100, epochs=100)


[CV 1/5; 1/15] END dropout_rate=0, learning_rate=0.001;, score=-4.724 total time=  18.7s
[CV 2/5; 1/15] START dropout_rate=0, learning_rate=0.001........................
[CV 2/5; 1/15] END dropout_rate=0, learning_rate=0.001;, score=-0.984 total time=  17.8s
[CV 3/5; 1/15] START dropout_rate=0, learning_rate=0.001........................
[CV 3/5; 1/15] END dropout_rate=0, learning_rate=0.001;, score=-2.056 total time=  17.8s
[CV 4/5; 1/15] START dropout_rate=0, learning_rate=0.001........................
[CV 4/5; 1/15] END dropout_rate=0, learning_rate=0.001;, score=-1.341 total time=  17.6s
[CV 5/5; 1/15] START dropout_rate=0, learning_rate=0.001........................
[CV 5/5; 1/15] END dropout_rate=0, learning_rate=0.001;, score=-1.820 total time=  18.1s
[CV 1/5; 2/15] START dropout_rate=0, learning_rate=0.01.........................
[CV 1/5; 2/15] END dropout_rate=0, learning_rate=0.01;, score=-2.104 total time=  20.6s
[CV 2/5; 2/15] START dropout_rate=0, learning_rate=0.01.......

[CV 4/5; 10/15] END dropout_rate=0.1, learning_rate=0.5;, score=-236.365 total time=  21.3s
[CV 5/5; 10/15] START dropout_rate=0.1, learning_rate=0.5.......................
[CV 5/5; 10/15] END dropout_rate=0.1, learning_rate=0.5;, score=-263.229 total time=  22.8s
[CV 1/5; 11/15] START dropout_rate=0.2, learning_rate=0.001.....................
[CV 1/5; 11/15] END dropout_rate=0.2, learning_rate=0.001;, score=-133.838 total time=  21.9s
[CV 2/5; 11/15] START dropout_rate=0.2, learning_rate=0.001.....................
[CV 2/5; 11/15] END dropout_rate=0.2, learning_rate=0.001;, score=-64.646 total time=  21.0s
[CV 3/5; 11/15] START dropout_rate=0.2, learning_rate=0.001.....................
[CV 3/5; 11/15] END dropout_rate=0.2, learning_rate=0.001;, score=-41.470 total time=  21.1s
[CV 4/5; 11/15] START dropout_rate=0.2, learning_rate=0.001.....................
[CV 4/5; 11/15] END dropout_rate=0.2, learning_rate=0.001;, score=-33.807 total time=  21.6s
[CV 5/5; 11/15] START dropout_rate=0.2

In [17]:
print('Best:{}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means= grid_result.cv_results_['mean_test_score']
stds= grid_result.cv_results_['std_test_score']
params= grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best:-1.4587859869003297, using {'dropout_rate': 0, 'learning_rate': 0.01}
-2.185088014602661,1.3228694079442718 with: {'dropout_rate': 0, 'learning_rate': 0.001}
-1.4587859869003297,0.42071364024718405 with: {'dropout_rate': 0, 'learning_rate': 0.01}
-120.19722337722779,136.4097707906779 with: {'dropout_rate': 0, 'learning_rate': 0.1}
-258.04595336914065,75.40702431378726 with: {'dropout_rate': 0, 'learning_rate': 0.2}
-258.6570251464844,65.37838584372786 with: {'dropout_rate': 0, 'learning_rate': 0.5}
-317.10897369384764,232.46627874394267 with: {'dropout_rate': 0.1, 'learning_rate': 0.001}
-40.492028427124026,21.652054164350496 with: {'dropout_rate': 0.1, 'learning_rate': 0.01}
-256.12334899902345,74.41667769546368 with: {'dropout_rate': 0.1, 'learning_rate': 0.1}
-256.3695770263672,72.82937562696078 with: {'dropout_rate': 0.1, 'learning_rate': 0.2}
-258.00766906738284,70.12188244984043 with: {'dropout_rate': 0.1, 'learning_rate': 0.5}
-122.3708740234375,113.4797580399232 with: {'dr

In [18]:
def create_model_af_ker_ini(activation_function,init):
  model= Sequential()
  model.add(Dense(10, input_dim=10, kernel_initializer= init, activation=activation_function))
  model.add(Dropout(0))
  model.add(Dense(8, kernel_initializer= init, activation=activation_function))
  model.add(Dropout(0))
  model.add(Dense(8, kernel_initializer= init, activation=activation_function))
  model.add(Dropout(0))
  model.add(Dense(1, kernel_initializer= init, activation='linear')) 
  adam= Adam(learning_rate= 0.001)
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model

In [19]:
model= KerasRegressor(build_fn=create_model_af_ker_ini, verbose=0, batch_size=100, epochs=100)
activation_function=['linear', 'relu']
init=['uniform', 'normal', 'zero']

param_grid= dict(activation_function= activation_function, init=init)
grid= GridSearchCV(estimator=model, param_grid=param_grid, cv= KFold(), verbose=10 )
grid_result= grid.fit(x,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START activation_function=linear, init=uniform....................


C:\Users\shiji\AppData\Local\Temp\ipykernel_17596\416632234.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model= KerasRegressor(build_fn=create_model_af_ker_ini, verbose=0, batch_size=100, epochs=100)


[CV 1/5; 1/6] END activation_function=linear, init=uniform;, score=-3.844 total time=  17.9s
[CV 2/5; 1/6] START activation_function=linear, init=uniform....................
[CV 2/5; 1/6] END activation_function=linear, init=uniform;, score=-0.945 total time=  18.3s
[CV 3/5; 1/6] START activation_function=linear, init=uniform....................
[CV 3/5; 1/6] END activation_function=linear, init=uniform;, score=-1.443 total time=  17.6s
[CV 4/5; 1/6] START activation_function=linear, init=uniform....................
[CV 4/5; 1/6] END activation_function=linear, init=uniform;, score=-3.687 total time=  17.5s
[CV 5/5; 1/6] START activation_function=linear, init=uniform....................
[CV 5/5; 1/6] END activation_function=linear, init=uniform;, score=-2.190 total time=  17.5s
[CV 1/5; 2/6] START activation_function=linear, init=normal.....................
[CV 1/5; 2/6] END activation_function=linear, init=normal;, score=-2.178 total time=  18.0s
[CV 2/5; 2/6] START activation_functio

In [20]:
print('Best:{}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means= grid_result.cv_results_['mean_test_score']
stds= grid_result.cv_results_['std_test_score']
params= grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best:-1.5054664015769958, using {'activation_function': 'linear', 'init': 'normal'}
-2.4218953251838684,1.1675513915425104 with: {'activation_function': 'linear', 'init': 'uniform'}
-1.5054664015769958,0.5107890334253626 with: {'activation_function': 'linear', 'init': 'normal'}
-15175.9724609375,820.9071688566831 with: {'activation_function': 'linear', 'init': 'zero'}
-2.872485029697418,1.8092893007515565 with: {'activation_function': 'relu', 'init': 'uniform'}
-2.3593665838241575,0.8907637361587024 with: {'activation_function': 'relu', 'init': 'normal'}
-15175.946484375,820.9031818203226 with: {'activation_function': 'relu', 'init': 'zero'}


In [21]:
def create_model_neuron(neuron1,neuron2, neuron3):
  model= Sequential()
  model.add(Dense(neuron1, input_dim=10, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(neuron2, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(neuron3, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(1, kernel_initializer= 'normal', activation='linear')) 
  adam= Adam(learning_rate= 0.001)
# Compile model
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model


In [22]:
model= KerasRegressor(build_fn=create_model_neuron, verbose=0, batch_size=100, epochs=100)
neuron1=[4,8,16]
neuron2=[4,8,16]
neuron3=[4,8,16]
param_grid= dict(neuron1= neuron1, neuron2=neuron2,neuron3=neuron3 )
grid= GridSearchCV(estimator=model, param_grid=param_grid, cv= KFold(), verbose=10 )
grid_result= grid.fit(x,y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5; 1/27] START neuron1=4, neuron2=4, neuron3=4............................


C:\Users\shiji\AppData\Local\Temp\ipykernel_17596\1206552891.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model= KerasRegressor(build_fn=create_model_neuron, verbose=0, batch_size=100, epochs=100)


[CV 1/5; 1/27] END neuron1=4, neuron2=4, neuron3=4;, score=-8.631 total time=  17.2s
[CV 2/5; 1/27] START neuron1=4, neuron2=4, neuron3=4............................
[CV 2/5; 1/27] END neuron1=4, neuron2=4, neuron3=4;, score=-1.105 total time=  17.6s
[CV 3/5; 1/27] START neuron1=4, neuron2=4, neuron3=4............................
[CV 3/5; 1/27] END neuron1=4, neuron2=4, neuron3=4;, score=-2.600 total time=  17.6s
[CV 4/5; 1/27] START neuron1=4, neuron2=4, neuron3=4............................
[CV 4/5; 1/27] END neuron1=4, neuron2=4, neuron3=4;, score=-1.984 total time=  18.8s
[CV 5/5; 1/27] START neuron1=4, neuron2=4, neuron3=4............................
[CV 5/5; 1/27] END neuron1=4, neuron2=4, neuron3=4;, score=-2.786 total time=  17.9s
[CV 1/5; 2/27] START neuron1=4, neuron2=4, neuron3=8............................
[CV 1/5; 2/27] END neuron1=4, neuron2=4, neuron3=8;, score=-4.091 total time=  17.6s
[CV 2/5; 2/27] START neuron1=4, neuron2=4, neuron3=8............................
[CV 

[CV 1/5; 11/27] END neuron1=8, neuron2=4, neuron3=8;, score=-2.885 total time=  17.7s
[CV 2/5; 11/27] START neuron1=8, neuron2=4, neuron3=8...........................
[CV 2/5; 11/27] END neuron1=8, neuron2=4, neuron3=8;, score=-0.768 total time=  18.0s
[CV 3/5; 11/27] START neuron1=8, neuron2=4, neuron3=8...........................
[CV 3/5; 11/27] END neuron1=8, neuron2=4, neuron3=8;, score=-1.803 total time=  17.7s
[CV 4/5; 11/27] START neuron1=8, neuron2=4, neuron3=8...........................
[CV 4/5; 11/27] END neuron1=8, neuron2=4, neuron3=8;, score=-2.716 total time=  17.6s
[CV 5/5; 11/27] START neuron1=8, neuron2=4, neuron3=8...........................
[CV 5/5; 11/27] END neuron1=8, neuron2=4, neuron3=8;, score=-1.430 total time=  17.8s
[CV 1/5; 12/27] START neuron1=8, neuron2=4, neuron3=16..........................
[CV 1/5; 12/27] END neuron1=8, neuron2=4, neuron3=16;, score=-2.441 total time=  18.9s
[CV 2/5; 12/27] START neuron1=8, neuron2=4, neuron3=16........................

[CV 5/5; 20/27] END neuron1=16, neuron2=4, neuron3=8;, score=-1.723 total time=  17.9s
[CV 1/5; 21/27] START neuron1=16, neuron2=4, neuron3=16.........................
[CV 1/5; 21/27] END neuron1=16, neuron2=4, neuron3=16;, score=-2.226 total time=  17.3s
[CV 2/5; 21/27] START neuron1=16, neuron2=4, neuron3=16.........................
[CV 2/5; 21/27] END neuron1=16, neuron2=4, neuron3=16;, score=-0.865 total time=  17.5s
[CV 3/5; 21/27] START neuron1=16, neuron2=4, neuron3=16.........................
[CV 3/5; 21/27] END neuron1=16, neuron2=4, neuron3=16;, score=-1.296 total time=  17.3s
[CV 4/5; 21/27] START neuron1=16, neuron2=4, neuron3=16.........................
[CV 4/5; 21/27] END neuron1=16, neuron2=4, neuron3=16;, score=-0.982 total time=  17.8s
[CV 5/5; 21/27] START neuron1=16, neuron2=4, neuron3=16.........................
[CV 5/5; 21/27] END neuron1=16, neuron2=4, neuron3=16;, score=-1.523 total time=  17.5s
[CV 1/5; 22/27] START neuron1=16, neuron2=8, neuron3=4..............

In [23]:
# summerize results
print('Best:{}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means= grid_result.cv_results_['mean_test_score']
stds= grid_result.cv_results_['std_test_score']
params= grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best:-1.3784620761871338, using {'neuron1': 16, 'neuron2': 4, 'neuron3': 16}
-3.421182084083557,2.670110067256335 with: {'neuron1': 4, 'neuron2': 4, 'neuron3': 4}
-2.0055784821510314,1.1692242389495255 with: {'neuron1': 4, 'neuron2': 4, 'neuron3': 8}
-2.8582144260406492,1.820606116082206 with: {'neuron1': 4, 'neuron2': 4, 'neuron3': 16}
-1.9190257668495179,1.1572747457925312 with: {'neuron1': 4, 'neuron2': 8, 'neuron3': 4}
-1.7762811422348022,0.9341411492925246 with: {'neuron1': 4, 'neuron2': 8, 'neuron3': 8}
-1.8093382716178894,0.8475854581238649 with: {'neuron1': 4, 'neuron2': 8, 'neuron3': 16}
-1.9224720478057862,0.7851378546660603 with: {'neuron1': 4, 'neuron2': 16, 'neuron3': 4}
-1.7871065855026245,0.4874594726166847 with: {'neuron1': 4, 'neuron2': 16, 'neuron3': 8}
-2.8988495469093323,2.8712730090307863 with: {'neuron1': 4, 'neuron2': 16, 'neuron3': 16}
-2.472635066509247,2.2172301299939607 with: {'neuron1': 8, 'neuron2': 4, 'neuron3': 4}
-1.9203901171684266,0.7934031233773167 wi